<a href="https://colab.research.google.com/github/kev457/COMP5349/blob/main/A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("COMP5349 A2") \
    .getOrCreate()

In [ ]:
test_data = "/content/drive/MyDrive/comp5349/test.json"
test_init_df = spark.read.json(test_data)
train_data = "/content/drive/MyDrive/comp5349/train_separate_questions.json"
train_init_df = spark.read.json(train_data)
all_data = "/content/drive/MyDrive/comp5349/CUADv1.json"
all_init_df = spark.read.json(all_data)

In [ ]:
test_init_df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- paragraphs: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- context: string (nullable = true)
 |    |    |    |    |-- qas: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |-- answers: array (nullable = true)
 |    |    |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |    |    |-- answer_start: long (nullable = true)
 |    |    |    |    |    |    |    |    |-- text: string (nullable = true)
 |    |    |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |    |    |-- is_impossible: boolean (nullable = true)
 |    |    |    |    |    |    |-- question: string (nullable = true)
 |    |    |-- title: string (nullable = true)
 |-- version: string (nullable = true)



In [ ]:
from pyspark.sql.functions import explode
test_data_df= test_init_df.select((explode("data").alias('data')))
test_paragraph_df = test_data_df.select(explode("data.paragraphs").alias("paragraph"))
qas_context_df = test_paragraph_df.select(explode("paragraph.qas").alias("qas"),"paragraph.context")
df = qas_context_df.select('qas.id','context','qas.question',explode("qas.answers").alias("answers"),'qas.is_impossible')

df.show()

+--------------------+--------------------+--------------------+--------------------+-------------+
|                  id|             context|            question|             answers|is_impossible|
+--------------------+--------------------+--------------------+--------------------+-------------+
|LohaCompanyltd_20...|Exhibit 10.16 SUP...|Highlight the par...|{14, SUPPLY CONTR...|        false|
|LohaCompanyltd_20...|Exhibit 10.16 SUP...|Highlight the par...|  {143, The seller:}|        false|
|LohaCompanyltd_20...|Exhibit 10.16 SUP...|Highlight the par...|{49, The buyer/En...|        false|
|LohaCompanyltd_20...|Exhibit 10.16 SUP...|Highlight the par...|{10985, The Contr...|        false|
|LohaCompanyltd_20...|Exhibit 10.16 SUP...|Highlight the par...|{10691, It will b...|        false|
|LohaCompanyltd_20...|Exhibit 10.16 SUP...|Highlight the par...|{6665, Within 7 d...|        false|
|LohaCompanyltd_20...|Exhibit 10.16 SUP...|Highlight the par...|{3659, To be cove...|        false|


In [ ]:
exploded_df  =  df.select('id','context', 'question', "answers.answer_start","answers.text",'is_impossible')
exploded_df.collect()[100]


Row(id='PACIRA PHARMACEUTICALS, INC. - A_R STRATEGIC LICENSING, DISTRIBUTION AND MARKETING AGREEMENT __Anti-Assignment', context='Exhibit 10.13\n\nConfidential Materials omitted and filed separately with the Securities and Exchange Commission. Asterisks denote omissions.\n\nDATED: OCTOBER 15, 2009\n\nPACIRA PHARMACEUTICALS, INC.\n\nand\n\nEKR THERAPEUTICS, INC.        AMENDED AND RESTATED   STRATEGIC LICENSING, DISTRIBUTION AND MARKETING AGREEMENT\n\n\n\n\n\nTHIS AMENDED AND RESTATED STRATEGIC LICENSING, DISTRIBUTION AND MARKETING AGREEMENT (the  "Agreement") is made on October 15, 2009 (the "Agreement Date") and is effective as of the Effective Date (as defined below), between:   PACIRA PHARMACEUTICALS, INC. (F/K/A SKYEPHARMA, INC.) a company incorporated in the state of California whose principal place of business is 10450 Sciences Center Drive, San Diego, California 92121 USA ("PPI"); and   EKR THERAPEUTICS, INC., a company incorporated in the state of Delaware whose principal place

In [ ]:
exploded_df.show()

+--------------------+--------------------+--------------------+------------+--------------------+-------------+
|                  id|             context|            question|answer_start|                text|is_impossible|
+--------------------+--------------------+--------------------+------------+--------------------+-------------+
|LohaCompanyltd_20...|Exhibit 10.16 SUP...|Highlight the par...|          14|     SUPPLY CONTRACT|        false|
|LohaCompanyltd_20...|Exhibit 10.16 SUP...|Highlight the par...|         143|         The seller:|        false|
|LohaCompanyltd_20...|Exhibit 10.16 SUP...|Highlight the par...|          49|The buyer/End-Use...|        false|
|LohaCompanyltd_20...|Exhibit 10.16 SUP...|Highlight the par...|       10985|The Contract is v...|        false|
|LohaCompanyltd_20...|Exhibit 10.16 SUP...|Highlight the par...|       10691|It will be govern...|        false|
|LohaCompanyltd_20...|Exhibit 10.16 SUP...|Highlight the par...|        6665|Within 7 days aft..

In [ ]:
def extraction(row):
  #Get the category name from the id, last element split by '_'
  category = row[0].split('_')[-1]

  context  = row[1]

  question  = row[2]

  answer_start = row[3]
  answer = row[4]
  if answer == None:
    answer_start = 0
    answer_end = 0
  else:
    answer_end = answer_start + len(answer)
  #contract segment
  slide = 4096
  stride = 2048
  context_len = len(context)
  if context_len%stride==0:
    seq_len = context_len/stride
  else:
    seq_len = context_len//stride + 1
  seq = []
  for i in range(seq_len):
    #(context[0:4096],0)(context[2048:6144],2048)..
    seq.append([context[i*stride:i*stride+slide],i*stride])

  


  is_impossible = row[5]

  return(category, question, answer, answer_start, answer_end, is_impossible,seq)

extracted_df = exploded_df.rdd.map(extraction)

In [ ]:
extracted_df.take(5)

[('Document Name',
  'Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract',
  'SUPPLY CONTRACT',
  14,
  29,
  False,
  [['Exhibit 10.16 SUPPLY CONTRACT Contract No: Date: The buyer/End-User: Shenzhen LOHAS Supply Chain Management Co., Ltd. ADD: Tel No. : Fax No. : The seller: ADD: The Contract is concluded and signed by the Buyer and Seller on , in Hong Kong. 1. General provisions 1.1 This is a framework agreement, the terms and conditions are applied to all purchase orders which signed by this agreement (hereinafter referred to as the "order"). 1.2 If the provisions of the agreement are inconsistent with the order, the order shall prevail. Not stated in order content will be subject to the provisions of agreement. Any modification, supplementary, give up should been written records, only to be valid by buyers and sellers authorized representative signature and confirmation, otherwise will be d

In [ ]:
def expand(row):
  for i in row[6]:
    return ((row[0],row[1],row[2],row[3],row[4],row[5]),i)

extracted_df1 = extracted_df.map(expand)


In [ ]:
extracted_df1.take(10)

[(('Document Name',
   'Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract',
   'SUPPLY CONTRACT',
   14,
   29,
   False),
  ['Exhibit 10.16 SUPPLY CONTRACT Contract No: Date: The buyer/End-User: Shenzhen LOHAS Supply Chain Management Co., Ltd. ADD: Tel No. : Fax No. : The seller: ADD: The Contract is concluded and signed by the Buyer and Seller on , in Hong Kong. 1. General provisions 1.1 This is a framework agreement, the terms and conditions are applied to all purchase orders which signed by this agreement (hereinafter referred to as the "order"). 1.2 If the provisions of the agreement are inconsistent with the order, the order shall prevail. Not stated in order content will be subject to the provisions of agreement. Any modification, supplementary, give up should been written records, only to be valid by buyers and sellers authorized representative signature and confirmation, otherwise wil

In [ ]:
# step 2

def step2(row):
  is_impossible = row[0][-1]

  if is_impossible == True:
    return(row[0][0],'impossible negative'),(row[1][0],row[0][1],row[0][2],0,0)
  else:
    answer_start = row[0][3]
    answer_end = row[0][4]
    seq_start = row[1][1]

    if answer_start - seq_start >=0 and answer_end - seq_start <=4096:
      return(row[0][0],'positive'),(row[1][0],row[0][1],row[0][2],answer_start - seq_start,answer_end - seq_start)
    else:
      return(row[0][0],'possible negative'),(row[1][0],row[0][1],row[0][2],0,0)


In [ ]:
labelled_df = extracted_df1.map(step2)

In [ ]:
labelled_df.take(5)

[(('Document Name', 'positive'),
  ('Exhibit 10.16 SUPPLY CONTRACT Contract No: Date: The buyer/End-User: Shenzhen LOHAS Supply Chain Management Co., Ltd. ADD: Tel No. : Fax No. : The seller: ADD: The Contract is concluded and signed by the Buyer and Seller on , in Hong Kong. 1. General provisions 1.1 This is a framework agreement, the terms and conditions are applied to all purchase orders which signed by this agreement (hereinafter referred to as the "order"). 1.2 If the provisions of the agreement are inconsistent with the order, the order shall prevail. Not stated in order content will be subject to the provisions of agreement. Any modification, supplementary, give up should been written records, only to be valid by buyers and sellers authorized representative signature and confirmation, otherwise will be deemed invalid. 2. The agreement and order 2.1 During the validity term of this agreement, The buyer entrust SHENZHEN YICHANGTAI IMPORT AND EXPORT TRADE CO., LTD or SHENZHEN LEHEY

In [ ]:

# For an impossible question in a contract, the number of impossible negative samples
# to keep equals the average number of positive samples of that question in other
# contracts that have at least one positive sample for the same question.

# For each contract, the number of possible negative samples to keep for each question
# equals the number of positive samples of that question in this contract.

count = labelled_df.countByKey()

def countPos(row):
  label = row[0][1]
  if label == 'positive':
    id = row[0][0]
    question = row[1][1]
    return((question,id),label)


pos_count = labelled_df.map(countPos).filter(lambda x: x is not None).groupByKey().mapValues(list).map(lambda x: (x[0],len(x[1]))).collect()
print(pos_count)

[(('Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract', 'Document Name'), 84), (('Highlight the parts (if any) of this contract related to "Insurance" that should be reviewed by a lawyer. Details: Is there a requirement for insurance that must be maintained by one party for the benefit of the counterparty?', 'Insurance'), 3), (('Highlight the parts (if any) of this contract related to "Agreement Date" that should be reviewed by a lawyer. Details: The date of the contract', 'Agreement Date'), 84), (('Highlight the parts (if any) of this contract related to "Non-Compete" that should be reviewed by a lawyer. Details: Is there a restriction on the ability of a party to compete with the counterparty or operate in a certain geography or business or technology sector?\xa0', 'Non-Compete'), 1), (('Highlight the parts (if any) of this contract related to "License Grant" that should be reviewed by a law

In [ ]:
def balanceResult(row):
  num_of_positive = count[row[0][0],"positive"]
  label = row[0][1]
  
  if label == 'possible negative':
    return row[0],row[1][:num_of_positive]
  elif label == 'impossible negative':
    question = row[1][2]
    id = row[0][0]
    total = []
    for i in pos_count:
      if i[0][0] == question and i[0][1]!=id:
        total.append(i[1])
    average = round(sum(total) / len(total))
    return row[0],row[1][:average]
  else:
    return row





group_key_df= labelled_df.groupByKey().mapValues(list).map(balanceResult).filter(lambda x: x is not None)
group_key_df.take(10)



[(('Document Name', 'positive'),
  [('Exhibit 10.16 SUPPLY CONTRACT Contract No: Date: The buyer/End-User: Shenzhen LOHAS Supply Chain Management Co., Ltd. ADD: Tel No. : Fax No. : The seller: ADD: The Contract is concluded and signed by the Buyer and Seller on , in Hong Kong. 1. General provisions 1.1 This is a framework agreement, the terms and conditions are applied to all purchase orders which signed by this agreement (hereinafter referred to as the "order"). 1.2 If the provisions of the agreement are inconsistent with the order, the order shall prevail. Not stated in order content will be subject to the provisions of agreement. Any modification, supplementary, give up should been written records, only to be valid by buyers and sellers authorized representative signature and confirmation, otherwise will be deemed invalid. 2. The agreement and order 2.1 During the validity term of this agreement, The buyer entrust SHENZHEN YICHANGTAI IMPORT AND EXPORT TRADE CO., LTD or SHENZHEN LEHE

In [128]:
# def getFinal(row):
#   for content in row[1]:
#     return (content[0],content[1],content[3],content[4])

final = group_key_df.flatMap(lambda x : x[1]).map(lambda x:(x[0],x[1],x[3],x[4])).toDF(["source", "question", "answer_start", "answer_end"])

final.printSchema()
final.show()
final.write.mode("Overwrite").json("output.json")

root
 |-- source: string (nullable = true)
 |-- question: string (nullable = true)
 |-- answer_start: long (nullable = true)
 |-- answer_end: long (nullable = true)

+--------------------+--------------------+------------+----------+
|              source|            question|answer_start|answer_end|
+--------------------+--------------------+------------+----------+
|Exhibit 10.16 SUP...|Highlight the par...|          14|        29|
|1                ...|Highlight the par...|         225|       251|
|Exhibit 10.13\n\n...|Highlight the par...|         322|       400|
|ENDORSEMENT\n\nCo...|Highlight the par...|          30|        41|
|Exhibit 10.4   IN...|Highlight the par...|          54|        85|
|Exhibit 10.4\n\nC...|Highlight the par...|          60|        98|
|EXHIBIT 10.13    ...|Highlight the par...|         505|       524|
|Exhibit 10.1\n\nS...|Highlight the par...|        1911|      1939|
|Exhibit 10.2    S...|Highlight the par...|          86|       114|
|EXHIBIT 10.2   EN